In [1]:
from load_data import *

In [5]:
path_to_file = "data/name.basics.tsv.gz"
df1 = load_to_dataframe(path_to_file)
df1.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,<NA>,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [8]:
df1.loc[:, 'isDirector'] = df1["primaryProfession"].astype(str).str.contains("director")

In [11]:
df1.loc[:, 'isActor'] = df1["primaryProfession"].astype(str).str.contains("actor") | df1["primaryProfession"].astype(str).str.contains("actress")

In [33]:
df1 = df1.loc[(df1["isDirector"] == True) | (df1["isActor"] == True)].drop(columns=["primaryProfession"])

In [34]:
df1

,nconst,primaryName,birthYear,deathYear,knownForTitles,isDirector,isActor
0,nm0000001,Fred Astaire,1899,1987,"tt0072308,tt0050419,tt0053137,tt0027125",False,True
1,nm0000002,Lauren Bacall,1924,2014,"tt0037382,tt0075213,tt0117057,tt0038355",False,True
2,nm0000003,Brigitte Bardot,1934,<NA>,"tt0057345,tt0049189,tt0056404,tt0054452",False,True
3,nm0000004,John Belushi,1949,1982,"tt0072562,tt0077975,tt0080455,tt0078723",False,True
4,nm0000005,Ingmar Bergman,1918,2007,"tt0050986,tt0083922,tt0050976,tt0069467",True,True
...,...,...,...,...,...,...,...
13583168,nm9993700,Sexy Angel,<NA>,<NA>,tt7523066,False,True
13583169,nm9993701,Sanjai Kuriakose,<NA>,<NA>,tt8736744,False,True
13583171,nm9993703,James Craigmyle,<NA>,<NA>,"tt11212278,tt10627062,tt6914160,tt6225166",False,True
13583176,nm9993708,Eli Bevins,<NA>,<NA>,"tt11702702,tt14508814,tt11772904,tt11772812",True,False


In [19]:
for i in df1.index:
    knownFor = df1.iloc[i].knownForTitles.split(",")
    stats = df2.loc[df2["tconst"].isin(knownFor)].drop(columns=["tconst"]).describe().loc[["count", "mean", "std"], :]
    

['tt0072308', 'tt0050419', 'tt0053137', 'tt0027125']

In [42]:
i = 0
knownFor = df1.iloc[i].knownForTitles.split(",")
stats = dict(df2.loc[df2["tconst"].isin(knownFor)].drop(columns=["tconst"]).describe().loc[["count", "mean", "std"], :])
stats

{'averageRating': count    4.00000
 mean     7.20000
 std      0.33665
 Name: averageRating, dtype: float64,
 'numVotes': count        4.000000
 mean     29264.750000
 std      15023.730482
 Name: numVotes, dtype: float64}

In [15]:
path_to_file = "data/title.ratings.tsv.gz"
df2 = load_to_dataframe(path_to_file)
df2.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.6,277
2,tt0000003,6.5,2024
3,tt0000004,5.4,179
4,tt0000005,6.2,2791
